# A5 小書產生器（v9）— 封面/內頁/封底皆支援**多張圖片** + **可見清單** + **排序/刪除/新增**
- 每個區塊（封面、內頁、封底）都可以上傳**多張**圖片
- 看得到**已上傳清單**，支援：**上移 / 下移 / 刪除單張 / 全清空**
- 可用 **URL 或 Base64** 新增圖片到清單（不依賴 FileUpload）
- 下載以 **Base64 超連結** 提供，避免 403/424
- 文字標記：`[14pt]`、`[圖片]` / `[圖片60%]`、`[左圖80%][右圖50%]`、`[無圖]`
- 內頁圖片按清單順序自動消耗；封面/封底會用清單的**第一張**（或你標記多張時依序消耗）

> 位置需求已調整：**圖片上傳與清單**就放在各自文字區塊**後方**（封面→封面圖、內頁→內頁圖、封底→封底圖）。


In [ ]:
# Imports
import io, re, base64, urllib.request, time, hashlib
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output

from docx import Document
from docx.shared import Mm, Pt, Inches
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx.oxml import OxmlElement
from docx.oxml.ns import qn

from PIL import Image


In [ ]:
# Helpers: bytes/Images, parsing, docx assembly

def ensure_rgb_jpeg(img_bytes, quality=90):
    im = Image.open(io.BytesIO(img_bytes))
    if im.mode not in ("RGB", "L"):
        im = im.convert("RGB")
    out = io.BytesIO()
    im.save(out, format="JPEG", quality=quality)
    return out.getvalue()

def parse_size_tag(text, default_pt=12):
    m = re.search(r"\[(\d+)pt\]\s*$", text)
    return int(m.group(1)) if m else default_pt

def strip_size_tag(text):
    return re.sub(r"\[\d+pt\]\s*$", "", text)

def has_skip_image_tag(text):
    return bool(re.search(r"\[無圖\]\s*$", text))

def strip_skip_image_tag(text):
    return re.sub(r"\[無圖\]\s*$", "", text)

def find_single_image_tag(text):
    m = re.search(r"\[(?:圖片)(\d+)?%?\]", text)
    if not m:
        return None
    pct = m.group(1)
    pct = int(pct) if pct else None
    return (pct, m.start(), m.end())

def find_double_image_tags(text):
    ml = re.search(r"\[左圖(\d+)?%?\]", text)
    mr = re.search(r"\[右圖(\d+)?%?\]", text)
    if not (ml and mr):
        return None
    lp = int(ml.group(1)) if ml.group(1) else None
    rp = int(mr.group(1)) if mr.group(1) else None
    return (lp, rp)

def set_section_to_a5(section):
    section.page_width = Mm(148)
    section.page_height = Mm(210)
    section.left_margin = Mm(15)
    section.right_margin = Mm(15)
    section.top_margin = Mm(15)
    section.bottom_margin = Mm(15)

def add_paragraph_with_size(doc, text, size_pt, align_center=False):
    p = doc.add_paragraph(text)
    p.alignment = WD_ALIGN_PARAGRAPH.CENTER if align_center else WD_ALIGN_PARAGRAPH.LEFT
    for run in p.runs:
        run.font.size = Pt(size_pt)
    return p

def add_single_image_paragraph(doc, img_bytes, width_pct=None):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    if width_pct is None:
        width_in = page_w_in * 0.9
    else:
        width_in = page_w_in * (width_pct / 100.0)
        width_in = max(1.0, min(page_w_in, width_in))
    safe = ensure_rgb_jpeg(img_bytes)
    doc.add_picture(io.BytesIO(safe), width=Inches(width_in))

def add_double_image_table(doc, left_bytes, right_bytes, l_pct, r_pct):
    page_w_mm = 148 - 15 - 15
    page_w_in = page_w_mm / 25.4
    base_in = page_w_in / 2.0
    l_in = base_in if l_pct is None else base_in * (l_pct / 100.0)
    r_in = base_in if r_pct is None else base_in * (r_pct / 100.0)
    l_in = max(1.0, min(page_w_in, l_in))
    r_in = max(1.0, min(page_w_in, r_in))

    table = doc.add_table(rows=1, cols=2)
    table.autofit = True
    cell = table.rows[0].cells[0]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(left_bytes)), width=Inches(l_in))
    cell = table.rows[0].cells[1]
    run = cell.paragraphs[0].add_run()
    run.add_picture(io.BytesIO(ensure_rgb_jpeg(right_bytes)), width=Inches(r_in))

def load_image_from_line(line: str) -> bytes:
    line = line.strip()
    if not line:
        return None
    if line.startswith('data:image'):
        head, b64 = line.split(',', 1)
        return base64.b64decode(b64)
    if line.startswith('http://') or line.startswith('https://'):
        with urllib.request.urlopen(line) as resp:
            return resp.read()
    # pure base64
    try:
        return base64.b64decode(line, validate=True)
    except Exception:
        raise ValueError("無法辨識的圖片輸入：請提供圖片 URL、data:base64 或純 base64。")

def make_data_uri_download(data: bytes, filename: str) -> HTML:
    b64 = base64.b64encode(data).decode()
    href = f'<a download="{filename}" href="data:application/vnd.openxmlformats-officedocument.wordprocessingml.document;base64,{b64}">📥 點此下載 {filename}</a>'
    return HTML(href)

def sha(b: bytes) -> str:
    import hashlib
    return hashlib.sha256(b).hexdigest()[:8]


In [ ]:
# UI builders: a reusable image manager widget (uploader + list + controls + URL/Base64 adder)

def image_manager(title: str, allow_many=True):
    uploader = widgets.FileUpload(accept='image/*', multiple=allow_many, description=f'上傳{title}')
    add_box = widgets.Textarea(placeholder='貼入圖片 URL 或 data:image/...;base64,... 或純 base64（每行一張）', layout=widgets.Layout(height='70px', width='100%'))
    add_btn = widgets.Button(description='新增到清單', button_style='info')
    lst = widgets.Select(options=[], rows=6, description='順序')
    btn_up = widgets.Button(description='上移')
    btn_down = widgets.Button(description='下移')
    btn_del = widgets.Button(description='刪除')
    btn_clear = widgets.Button(description='清空')
    status = widgets.HTML('')
    
    images = []  # list of dict: {'name', 'bytes', 'id'}
    
    def refresh():
        lst.options = [f"{i+1:02d}. {it['name']} ({len(it['bytes'])//1024}KB)" for i,it in enumerate(images)]
        if images:
            lst.index = 0 if lst.index is None else min(lst.index, len(images)-1)
        else:
            lst.index = None
    
    def on_upload(change):
        nonlocal images
        for meta in uploader.value:
            name = meta.get('name','image')
            content = meta.get('content', b'')
            if content:
                images.append({'name': name, 'bytes': content, 'id': sha(content)})
        uploader.value.clear()
        refresh()
        status.value = f"<span style='color:green'>已加入 {len(images)} 張</span>"
    
    uploader.observe(on_upload, names='value')
    
    def do_add_from_text(_):
        nonlocal images
        lines = [ln.strip() for ln in add_box.value.splitlines() if ln.strip()]
        added = 0
        for ln in lines:
            try:
                b = load_image_from_line(ln)
                if b:
                    images.append({'name': f'added_{sha(b)}.jpg', 'bytes': b, 'id': sha(b)})
                    added += 1
            except Exception:
                pass
        add_box.value = ''
        refresh()
        status.value = f"<span style='color:green'>新增 {added} 張</span>" if added else "<span style='color:#a00'>沒有成功新增</span>"
    
    add_btn.on_click(do_add_from_text)
    
    def move(delta):
        if not images or lst.index is None: return
        i = lst.index; j = i + delta
        if 0 <= j < len(images):
            images[i], images[j] = images[j], images[i]
            refresh(); lst.index = j
    
    btn_up.on_click(lambda _: move(-1))
    btn_down.on_click(lambda _: move(1))
    
    def do_delete(_):
        if not images or lst.index is None: return
        images.pop(lst.index); refresh()
    btn_del.on_click(do_delete)
    
    def do_clear(_):
        images.clear(); refresh()
    btn_clear.on_click(do_clear)
    
    ui = widgets.VBox([
        uploader,
        widgets.HBox([lst, widgets.VBox([btn_up, btn_down, btn_del, btn_clear])]),
        widgets.HBox([add_box, widgets.VBox([add_btn])]),
        status
    ])
    
    return ui, images, lst

# Text areas
txt_cover = widgets.Textarea(value="我的 A5 小書\n[圖片60%]", description='封面文字', layout=widgets.Layout(width='100%', height='90px'))
txt_body = widgets.Textarea(value="第一章……\n[圖片]\n第二章……\n[左圖80%][右圖60%]\n", description='內頁文字', layout=widgets.Layout(width='100%', height='180px'))
txt_back = widgets.Textarea(value="封底簡介……\n[圖片60%]", description='封底文字', layout=widgets.Layout(width='100%', height='90px'))
chk_pagenum = widgets.Checkbox(value=True, description='內頁加頁碼')

# Managers placed right after each text block
ui_cover, imgs_cover, _ = image_manager("封面圖片（可多張）", allow_many=True)
ui_body, imgs_body, _  = image_manager("內頁圖片（可多張）", allow_many=True)
ui_back, imgs_back, _  = image_manager("封底圖片（可多張）", allow_many=True)

btn = widgets.Button(description="生成 A5 小書（docx）", button_style='success')
out = widgets.Output()

display(widgets.VBox([
    widgets.HBox([widgets.VBox([txt_cover]), widgets.VBox([chk_pagenum])]),
    ui_cover,
    widgets.Label('—— 內頁 ——'),
    txt_body,
    ui_body,
    widgets.Label('—— 封底 ——'),
    txt_back,
    ui_back,
    btn,
    out
]))


In [ ]:
# Assemble DOCX using the image manager lists

def build_docx_bytes():
    cover_text = txt_cover.value
    body_text  = txt_body.value
    back_text  = txt_back.value
    
    # collect bytes per section in current order
    cover_imgs = [it['bytes'] for it in imgs_cover]
    body_imgs  = [it['bytes']  for it in imgs_body]
    back_imgs  = [it['bytes']  for it in imgs_back]
    
    doc = Document()
    set_section_to_a5(doc.sections[0])
    
    # Cover
    size_pt = parse_size_tag(cover_text, default_pt=18)
    text = strip_size_tag(cover_text)
    single = find_single_image_tag(text)
    auto_used = 0
    if single and len(cover_imgs) > 0:
        pct, s, e = single
        text_clean = (text[:s] + text[e:]).strip()
        if text_clean:
            add_paragraph_with_size(doc, text_clean, size_pt, align_center=True)
        add_single_image_paragraph(doc, cover_imgs[0], width_pct=pct)
        auto_used = 1
    else:
        add_paragraph_with_size(doc, text.strip(), size_pt, align_center=True)
        # 沒標記也自動放第一張
        if cover_imgs:
            add_single_image_paragraph(doc, cover_imgs[0], width_pct=60)
            auto_used = 1
    # 若文字中其實放了多個 [圖片]，內文邏輯才會消耗多張；封面這邊只用第 1 張作預設
    
    # Body
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    if chk_pagenum.value:
        add_page_number_footer(doc.sections[-1], start_from=1)
    
    paragraphs = [p.strip() for p in body_text.split('\n')]
    img_idx = 0
    for raw in paragraphs:
        if not raw:
            doc.add_paragraph(""); continue
        size_pt = parse_size_tag(raw, default_pt=12)
        text = strip_size_tag(raw)
        skip_img = has_skip_image_tag(text)
        text = strip_skip_image_tag(text)
        
        dbl = find_double_image_tags(text)
        if dbl and (not skip_img) and (img_idx + 1 < len(body_imgs)):
            text_clean = re.sub(r"\[左圖\d*%?\]\s*\[右圖\d*%?\]", "", text).strip()
            if text_clean: add_paragraph_with_size(doc, text_clean, size_pt)
            l_pct, r_pct = dbl
            add_double_image_table(doc, body_imgs[img_idx], body_imgs[img_idx+1], l_pct, r_pct)
            img_idx += 2; continue
        
        single = find_single_image_tag(text)
        if single and (not skip_img) and (img_idx < len(body_imgs)):
            pct, s, e = single
            text_clean = (text[:s] + text[e:]).strip()
            if text_clean: add_paragraph_with_size(doc, text_clean, size_pt)
            add_single_image_paragraph(doc, body_imgs[img_idx], width_pct=pct)
            img_idx += 1; continue
        
        add_paragraph_with_size(doc, text, size_pt)
    
    # Back
    doc.add_section()
    set_section_to_a5(doc.sections[-1])
    size_pt = parse_size_tag(back_text, default_pt=12)
    text = strip_size_tag(back_text)
    single = find_single_image_tag(text)
    if single and back_imgs:
        pct, s, e = single
        text_clean = (text[:s] + text[e:]).strip()
        if text_clean: add_paragraph_with_size(doc, text_clean, size_pt, align_center=True)
        add_single_image_paragraph(doc, back_imgs[0], width_pct=pct)
    else:
        add_paragraph_with_size(doc, text.strip(), size_pt, align_center=True)
        if back_imgs:
            add_single_image_paragraph(doc, back_imgs[0], width_pct=60)
    
    buf = io.BytesIO(); doc.save(buf); data = buf.getvalue()
    _ = Document(io.BytesIO(data))  # verify
    return data

def on_click(_):
    with out:
        clear_output()
        try:
            data = build_docx_bytes()
            fname = f"a5_book_{int(time.time())}.docx"
            display(HTML('<div style="color:green;">完成！</div>'))
            display(make_data_uri_download(data, fname))
        except Exception as e:
            display(HTML(f'<div style="color:red;">失敗：{e}</div>'))

btn.on_click(on_click)
